In [7]:
"""
This heuristic-based code provides a solution for matching students into groups based on their preferences and skill levels for team formation. It includes the following components:

1. A Student class to store student information such as user ID, name, current group size, desired group size, skill levels, availability, and timestamp.

2. A match_students function that takes a list of students and the maximum group size as input. It iterates through the students, finds potential matches based on 
   skill level compatibility and availability, and forms complete or incomplete groups accordingly. 
   The function also handles unassigned students and attempts to add them to incomplete groups at different availability thresholds.

3. Helper functions:
   - find_potential_matches: Finds potential matches for a student based on skill level compatibility, availability, and group size constraints.
   - find_potential_additions: Finds potential additions to an existing group based on skill level compatibility, availability, and group size constraints.
   - create_group: Creates a string representation of a group, including group members, average skill level, and common availability.
   - calculate_average_skill_level: Calculates the average skill level when combining two students' groups.
   - calculate_average_skill_level_group: Calculates the average skill level for a group of students.
   - check_availability: Checks if two students have at least two common availability slots.
   - check_availability_group: Checks if a group meets the availability threshold based on the minimum required common availability slots.
   - find_common_availability: Finds the common availability slots for a group of students.

The code also includes a sample dataset of 15 students with various attributes for testing purposes.
You can load the actual data collected from users and apply this approach and functions.

Usage:
1. Import the Student class and the match_students function.
2. Create a list of Student objects with the required attributes.
3. Call the match_students function with the list of students and the desired maximum group size.
4. The function returns three lists: formed_groups (complete groups), incomplete_groups (partially formed groups), and unassigned_users (students who could not be matched).
5. Process the returned lists according to your application's requirements.
"""


#data structure to store student information

class Student:
    def __init__(self, user_id, name, current_group_size, desired_group_size, skill_levels, availability, timestamp):
        self.user_id = user_id
        self.name = name
        self.current_group_size = current_group_size
        self.desired_group_size = desired_group_size
        self.skill_levels = skill_levels
        self.availability = availability
        self.timestamp = timestamp
        self.assigned_group = None
        
#function to match students 
#takes the student to be matches and max group size needed

def match_students(students, max_group_size):
    formed_groups = []
    incomplete_groups = []
    unassigned_users = {}

    sorted_students = sorted(students, key=lambda x: x.timestamp) #preference to students who signed up sooner

    for student in sorted_students:
        if student.assigned_group is not None:
            continue

        potential_matches = find_potential_matches(student, sorted_students, max_group_size)

        if potential_matches:
            matched_students = [student]
            matched_students.extend(potential_matches[:student.desired_group_size - student.current_group_size])
            new_group = create_group(matched_students, len(formed_groups) + len(incomplete_groups) + 1)

            if len(matched_students) == student.desired_group_size:
                formed_groups.append(new_group)
#                 print(f"Formed complete group: {new_group}") #uncomment this part to print intermediate outputs
            else:
                incomplete_groups.append(matched_students)
#                 print(f"Formed incomplete group: {new_group}")

            for matched_student in matched_students:
                matched_student.assigned_group = new_group
        else:
            unassigned_users[student.user_id] = student
#             print(f"No potential matches found for student: {student.user_id}")

    availability_thresholds = [1.0, 0.75, 0.5]

    for threshold in availability_thresholds:
#         print(f"\nIntermediate groupings at availability threshold {threshold}:")
        for group in incomplete_groups:
            potential_additions = find_potential_additions(group, sorted_students, max_group_size, threshold)

            if potential_additions:
                for addition in potential_additions:
                    if addition.assigned_group is None and len(group) < group[0].desired_group_size:
                        group.append(addition)
                        addition.assigned_group = create_group(group, len(formed_groups) + len(incomplete_groups) + 1)
#                         print(f"Added student {addition.user_id} to group: {create_group(group, len(formed_groups) + len(incomplete_groups) + 1)}") #intermediate output
                    else:
                        break

            if len(group) == group[0].desired_group_size:
                formed_groups.append(create_group(group, len(formed_groups) + len(incomplete_groups) + 1))
                incomplete_groups.remove(group)
#                 print(f"Completed group: {create_group(group, len(formed_groups) + len(incomplete_groups) + 1)}")

    return formed_groups, incomplete_groups, unassigned_users

def find_potential_matches(student, students, max_group_size):
    potential_matches = []

    for other_student in students:
        if other_student.assigned_group is not None or other_student.user_id == student.user_id:
            continue

        if student.current_group_size + other_student.current_group_size <= max_group_size:
            if calculate_average_skill_level(student, other_student) >= 2:
                if check_availability(student, other_student):
                    potential_matches.append(other_student)

    return potential_matches

def find_potential_additions(group, students, max_group_size, availability_threshold):
    potential_additions = []

    for student in students:
        if student.assigned_group is not None:
            continue

        if len(group) + student.current_group_size <= max_group_size:
            if calculate_average_skill_level_group(group + [student]) >= 2:
                if check_availability_group(group + [student], availability_threshold):
                    potential_additions.append(student)

    return potential_additions

def create_group(students, group_id):
    group_name = f"Group {group_id}"
    group_members = [f"{student.name} (ID: {student.user_id}, Skill Levels: {student.skill_levels})" for student in students]
    avg_skill_level = calculate_average_skill_level_group(students)
    common_availability = find_common_availability(students)

    group_details = f"{group_name}:\n"
    group_details += f"  Members: {', '.join(group_members)}\n"
    group_details += f"  Average Skill Level: {avg_skill_level:.2f}\n"
    group_details += f"  Common Availability: {', '.join(common_availability)}"

    return group_details

#here student 1 represents the average skill of group, as that is what is collected through online form
#1 sign up caters to partial team

def calculate_average_skill_level(student1, student2):
    total_students = student1.current_group_size + student2.current_group_size
    
    # Calculate the sum of all skill levels for student1's group
    student1_total_skills = sum(student1.skill_levels)
    
    # Calculate the sum of all skill levels for student2's group
    student2_total_skills = sum(student2.skill_levels)
    
    total_skills = student1_total_skills + student2_total_skills
    
    return total_skills / total_students

def calculate_average_skill_level_group(group):
    skill_levels = [student.skill_levels for student in group]
    return sum(sum(levels) for levels in skill_levels) / sum(len(levels) for levels in skill_levels)

def check_availability(student1, student2):
    common_availability = set(student1.availability) & set(student2.availability)
    return len(common_availability) >= 2

def check_availability_group(group, threshold):
    availabilities = [set(student.availability) for student in group]
    common_availability = set.intersection(*availabilities)
    return len(common_availability) >= 2 and len(group) * threshold <= len(common_availability) * len(group)

def find_common_availability(group):
    availabilities = [set(student.availability) for student in group]
    common_availability = set.intersection(*availabilities)
    return list(common_availability)

# sample dataset with 15 people
students = [
    Student(1, "Alice", 2, 4, [3, 2, 3], ["Mon", "Tue", "Wed"], "2023-06-01 10:00:00"),
    Student(2, "Bob", 1, 3, [2, 3, 2], ["Tue", "Wed", "Thu"], "2023-06-01 11:00:00"),
    Student(3, "Charlie", 1, 2, [3, 3, 3], ["Mon", "Tue", "Fri"], "2023-06-01 12:00:00"),
    Student(4, "David", 2, 3, [2, 2, 2], ["Wed", "Thu"], "2023-06-01 13:00:00"),
    Student(5, "Eve", 1, 3, [2, 3, 3], ["Tue", "Wed"], "2023-06-01 14:00:00"),
    Student(6, "Frank", 1, 2, [2, 2, 2], ["Mon", "Wed"], "2023-06-01 15:00:00"),
    Student(7, "Grace", 1, 4, [3, 2, 3], ["Tue", "Thu", "Fri"], "2023-06-01 16:00:00"),
    Student(8, "Henry", 2, 3, [2, 3, 2], ["Mon", "Wed", "Fri"], "2023-06-01 17:00:00"),
    Student(9, "Ivy", 1, 2, [3, 3, 3], ["Tue", "Wed", "Thu"], "2023-06-01 18:00:00"),
    Student(10, "Jack", 1, 3, [2, 2, 2], ["Mon", "Tue", "Fri"], "2023-06-01 19:00:00"),
    Student(11, "Kate", 1, 4, [2, 3, 3], ["Wed", "Thu", "Fri"], "2023-06-01 20:00:00"),
    Student(12, "Liam", 2, 2, [2, 2, 2], ["Mon", "Tue", "Wed"], "2023-06-01 21:00:00"),
    Student(13, "Mia", 1, 3, [3, 2, 3], ["Tue", "Thu", "Fri"], "2023-06-01 22:00:00"),
    Student(14, "Noah", 1, 4, [2, 3, 2], ["Mon", "Wed", "Thu"], "2023-06-01 23:00:00"),
    Student(15, "Olivia", 1, 3, [3, 3, 3], ["Tue", "Wed", "Fri"], "2023-06-01 24:00:00")
]

max_group_size = 4

formed_groups, incomplete_groups, unassigned_users = match_students(students, max_group_size)

print("\nFinal complete groups:")
for group in formed_groups:
    print(group)

print("\nIncomplete groups:")
for group in incomplete_groups:
    print(create_group(group, len(formed_groups) + len(incomplete_groups) + 1))

print("\nUnassigned users:")
for user_id in unassigned_users:
    print(f"User ID: {user_id}")
    
#tabular format print output of formed groups 

# print("User ID | Group ID | Status                | Avg Skill Level | Common Availability | Desired Group Size Match Count")
# print("--------|----------|----------------------|-----------------|---------------------|-------------------------------")

# for group in formed_groups:
#     for member in group["members"]:
#         print(f"{member.user_id:7} | {group['group_id']:8} | Final group formed   | {group['avg_skill_level']:.2f}             | {', '.join(group['common_availability']):20} | {group['desired_group_size_match_count']:30}")

# for group in incomplete_groups:
#     for member in group:
#         print(f"{member.user_id:7} | {member.assigned_group['group_id']:8} | Incomplete group     | {member.assigned_group['avg_skill_level']:.2f}             | {', '.join(member.assigned_group['common_availability']):20} | {member.assigned_group['desired_group_size_match_count']:30}")

# for user_id, student in unassigned_users.items():
#     print(f"{user_id:7} | {'N/A':8} | Unassigned user      | {'N/A':15} | {'N/A':20} | {'N/A':30}")

SyntaxError: f-string: unterminated string (2779186324.py, line 221)